<a href="https://colab.research.google.com/github/fsilvao/ia/blob/main/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm


#Configuración
epochs = 100
batch_size = 64
sample_size = 100 # Número de valores aleatorios para muestrear
g_lr= 1.0e-4
d_lr= 1.0e-4

#Dataloader para MNIST

transform = transforms.ToTensor()
dataset = datasets.MNIST(root='./data', train = True, download = True, transform = transform)
dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True)

#Red generadora
class Generator(nn.Sequential):
  def __init__(self, sample_size:int):
    super().__init__(
        nn.Linear(sample_size, 128),
        nn.LeakyReLU(0.01),
        nn.Linear(128, 784),
        nn.Sigmoid()
    )
    #Tamaño del vector de valor aleatorio
    self.sample_size = sample_size

  def forward(self, batch_size: int):
    #Generamos valores aleatorios
    z = torch.randn(batch_size, self.sample_size)

    #Salida del generador
    output = super().forward(z)

    #Convertimos el output a una escala de grises (1x28x28)
    generated_images = output.reshape(batch_size, 1,28,28)
    return generated_images

#Red Discriminadora
class Discriminator(nn.Sequential):
  def __init__(self):
    super().__init__(
        nn.Linear(784, 128),
        nn.LeakyReLU(0.01),
        nn.Linear(128,1)
    )

  def forward(self, images: torch.Tensor, targets: torch.Tensor):
    prediction = super().forward(images.reshape(-1,784))
    loss = F.binary_cross_entropy_with_logits(prediction,targets)
    return loss

#Función auxiliar para guardar imágenes en forma de grilla:

def save_image_grid(epoch:int, images: torch.Tensor, ncol:int):
  image_grid = make_grid(images, ncol) #Imágenes en la grilla
  image_grid = image_grid.permute(1, 2, 0) #Move channel last
  image_grid = image_grid.cpu().numpy() #A numpy

  plt.imshow(image_grid)
  plt.xticks([])
  plt.yticks([])
  plt.savefig(f'generated_{epoch:03d}.jpg')
  plt.close()


#Etiquetas reales y falsas/generadas
real_targets = torch.ones(batch_size, 1)
fake_targets = torch.zeros(batch_size, 1)

#Instanciamos las redes generadora y discriminadora
generator = Generator(sample_size)
discriminator = Discriminator()

#Optimizadores
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=d_lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=g_lr)

#Loop de entrenamiento
for epoch in range(epochs):
  d_losses = []
  g_losses = []

  for images, labels in tqdm(dataloader):
    #Entrenamos el discriminador
    #Loss con las imágenes MNIST como inputs y etiquetas real_targets
    discriminator.train()
    d_loss = discriminator(images, real_targets)

    #Generar imágenes en modo eval
    generator.eval()
    with torch.no_grad():
      generated_images = generator(batch_size)

    # Loss con las imágenes generads como inputs y etiquetas fake_targets
    d_loss += discriminator(generated_images, fake_targets)

    #Optimizamos los parámetros del discriminador
    d_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()


    #Entrenamos el Generador
    generator.train()
    generated_images = generator(batch_size)

    #Loss con imágenes generadas como input y etiquetas real_targets
    discriminator.eval()
    g_loss = discriminator(generated_images, real_targets)

    #Optimizamos los parámetros del generador:
    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()

    #Guardamos los losses para el log
    d_losses.append(d_loss.item())
    g_losses.append(g_loss.item())

  #Imprimir losses en promedio
  print(epoch, np.mean(d_losses), np.mean(g_losses))

  #Guardamos las imágenes
  save_image_grid(epoch, generator(batch_size), ncol=8)



100%|██████████| 9912422/9912422 [00:00<00:00, 80256877.93it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 61490199.91it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 16721086.15it/s]


...

100%|██████████| 937/937 [00:18<00:00, 49.95it/s]


82 0.6524546313056824 2.1354983800757785


100%|██████████| 937/937 [00:18<00:00, 50.25it/s]


83 0.6489339548788304 2.1152512944495285


 83%|████████▎ | 781/937 [00:14<00:02, 55.67it/s]